# Projet IA01

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd


heart = pd.read_csv("heart.csv")


# Analyse exploratoire des données

In [ ]:
#distribution des variables catégorielles
plt.figure()
sns.histplot(x='Sex', data=heart)
plt.title('Sex distribution')
plt.show()

plt.figure()
sns.histplot(x='ChestPainType', data=heart)
plt.title('ChestPainType distribution')
plt.show()

plt.figure()
sns.histplot(x='RestingECG', data=heart)
plt.title('RestingECG distribution')
plt.show()

plt.figure()
sns.histplot(x='RestingECG', data=heart)
plt.title('RestingECG distribution')
plt.show()

plt.figure()
sns.histplot(x='ExerciseAngina', data=heart)
plt.title('ExerciseAngina distribution')
plt.show()

plt.figure()
sns.histplot(x='ST_Slope', data=heart)
plt.title('ST Slope distribution')
plt.show()


In [ ]:
#les variables catégorielles ont-elles un impact sur HeartDisease ?

matrice_corr = heart.corr(numeric_only=True)
sns.heatmap(matrice_corr, annot=True)
plt.show()

sns.countplot(x=heart['Sex'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['ChestPainType'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['RestingECG'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['ExerciseAngina'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['ST_Slope'], hue=heart['HeartDisease'])
plt.show()

sns.countplot(x=heart['FastingBS'], hue=heart['HeartDisease'])
plt.show()

In [ ]:
#identification des valeurs manquantes et aberrantes :

heart['FastingBS'] = heart['FastingBS'].astype(object)
heart['HeartDisease'] = heart['HeartDisease'].astype(object)

print("Number of Duplicates:", heart.duplicated().sum(), "\n") #nombre de doublon
print("Nombre de NaN : \n" , heart.isna().sum()) #nombre de données manquantes
    

num_df = heart.select_dtypes(include=['int64', 'float64'])

min_values = num_df.describe().loc['min']

print("Minimums des colonnes numeriques : ")
print(min_values)

cat_df = heart.select_dtypes(include=['object'])

print("Valeurs uniques des variables catégorielles : \n")
for col in cat_df.columns :
    unique_value = cat_df[col].unique()
    print(unique_value)

sns.boxplot(y=heart['Age'])
plt.title("Boxplot de l'âge")
plt.show()

sns.boxplot(y=heart['Cholesterol'])
plt.title("Boxplot du Cholesterol")
plt.show()
    
sns.boxplot(y=heart['RestingBP'])
plt.title("Boxplot de la tension")
plt.show()

sns.boxplot(y=heart['MaxHR'])
plt.title("Boxplot du maximum de fréquence cardiaque atteind")
plt.show()

print("c'est bon ça marche")

### Traitons maintenant les données abbérrantes